In [39]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import urllib
import json
import pickle
import re
import tqdm
from socket import timeout
import logging
import glob

from threading import Thread
from collections import defaultdict

%matplotlib inline

In [2]:
df = pd.read_csv("../datas/prepared/final_dataset.csv", index_col=0, encoding="ISO-8859-1")

In [3]:
df.head()

,primaryTitle,originalTitle,startYear,runtimeMinutes,averageRating,numVotes,Action,Adventure,Animation,Biography,...,Director_6,Director_7,Writer_0,Writer_1,Writer_2,Writer_3,Writer_4,Writer_5,Writer_6,Writer_7
tconst,,,,,,,,,,,,,,,,,,,,,
tt0002423,Madame DuBarry,Madame DuBarry,1919.0,85,6.9,600,0,0,0,1,...,1,0,1,0,0,0,0,0,0,1
tt0003037,Fantômas: Juve versus Fantômas,Juve contre Fantômas,1913.0,61,6.8,1126,0,0,0,0,...,1,0,1,0,0,0,0,0,0,1
tt0003165,Fantômas: The Dead Man Who Killed,Le mort qui tue,1913.0,90,7.0,861,0,0,0,0,...,1,0,1,0,0,0,0,0,0,1
tt0003643,The Avenging Conscience: or 'Thou Shalt Not Kill',The Avenging Conscience: or 'Thou Shalt Not Kill',1914.0,78,6.9,939,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
tt0003952,Fantômas: The False Magistrate,Le faux magistrat,1914.0,70,6.9,764,0,0,0,0,...,1,0,1,0,0,0,0,0,0,1


# Check API

In [20]:
url = "http://www.omdbapi.com/?i=tt3896198&apikey=8eec8978"
response = urllib.request.urlopen(url)
content = response.read()
parsed_content = json.loads(content)
parsed_content

In [23]:
parsed_content

{'Actors': 'Chris Pratt, Zoe Saldana, Dave Bautista, Vin Diesel',
 'Awards': 'Nominated for 1 Oscar. Another 10 wins & 30 nominations.',
 'BoxOffice': '$389,804,217',
 'Country': 'USA, New Zealand, Canada',
 'DVD': '22 Aug 2017',
 'Director': 'James Gunn',
 'Genre': 'Action, Adventure, Sci-Fi',
 'Language': 'English',
 'Metascore': '67',
 'Plot': "The Guardians must fight to keep their newfound family together as they unravel the mystery of Peter Quill's true parentage.",
 'Poster': 'https://images-na.ssl-images-amazon.com/images/M/MV5BMTg2MzI1MTg3OF5BMl5BanBnXkFtZTgwNTU3NDA2MTI@._V1_SX300.jpg',
 'Production': 'Walt Disney Pictures',
 'Rated': 'PG-13',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.7/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '83%'},
  {'Source': 'Metacritic', 'Value': '67/100'}],
 'Released': '05 May 2017',
 'Response': 'True',
 'Runtime': '136 min',
 'Title': 'Guardians of the Galaxy Vol. 2',
 'Type': 'movie',
 'Website': 'https://marvel.com/guard

In [25]:
url = "http://www.omdbapi.com/?i=tt0007867&apikey=8eec8978"
response = urllib.request.urlopen(url)
content2 = response.read()
parsed_content2 = json.loads(content2)
parsed_content2

{'Actors': 'Dorothy Phillips, William Stowell, Lon Chaney, Sydney Deane',
 'Awards': 'N/A',
 'BoxOffice': 'N/A',
 'Country': 'USA',
 'DVD': 'N/A',
 'Director': 'Joseph De Grasse',
 'Genre': 'Drama',
 'Language': 'English',
 'Metascore': 'N/A',
 'Plot': "Nora Helmer has years earlier committed a forgery in order to save the life of her authoritarian husband Torvald. Now she is being blackmailed lives in fear of her husband's finding out and...",
 'Poster': 'N/A',
 'Production': 'N/A',
 'Rated': 'N/A',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.2/10'}],
 'Released': '11 Jun 1917',
 'Response': 'True',
 'Runtime': 'N/A',
 'Title': "A Doll's House",
 'Type': 'movie',
 'Website': 'N/A',
 'Writer': 'Joseph De Grasse, Henrik Ibsen (play)',
 'Year': '1917',
 'imdbID': 'tt0007867',
 'imdbRating': '7.2',
 'imdbVotes': '13'}

# Extraction

The private key is stored in a file not shared. A logger will be set in order to stored timeout movies and run them later on

In [5]:
with open("private.txt", "r") as f:
    key = f.read()

In [6]:
logger = logging.getLogger('extractor')
hdlr = logging.FileHandler('extration.log')
logger.addHandler(hdlr) 
logger.setLevel(logging.WARNING)

In [34]:
def extract_content(movie_key, private_key, timeout_ = 10):
    url = "http://www.omdbapi.com/?i="+movie_key+"&apikey="+private_key
    try:
        response = urllib.request.urlopen(url, timeout = timeout_).read()
    except (urllib.error.HTTPError, urllib.error.URLError) as error:
        logger.error('Data of %s not retrieved because %s', movie_key, error)
        return -1
    except timeout:
        logger.error('socket timed out - Key %s', movie_key)
        return -1
    return json.loads(response)

In [8]:
n = len(df.index.values)
datas = []
start = 0
for i, tconst in enumerate(df.index.values):
    if i < start : 
        continue
    if i > start and i % 1000 == 0:
        df2 = pd.DataFrame.from_dict(datas)
        df2.to_csv("../datas/raw/omdb_{}.csv".format(i//1000), index=False)
        datas = []
    if i % 100 == 0 :
        p = 100*i/n
        print("{:.2f} %".format(p), sep="")
    content = extract_content(tconst, key, 3)
    if content != -1 and content["Response"] == "True":
        datas.append(content)
print("Done")
df2 = pd.DataFrame.from_dict(datas)
df2.to_csv("../datas/raw/omdb_end.csv", index=False)

0.00 %
0.25 %
0.49 %
0.74 %
0.99 %
1.23 %
1.48 %
1.73 %
1.98 %
2.22 %
2.47 %
2.72 %
2.96 %
3.21 %
3.46 %
3.70 %
3.95 %
4.20 %
4.44 %
4.69 %
4.94 %
5.19 %
5.43 %
5.68 %
5.93 %
6.17 %
6.42 %
6.67 %
6.91 %
7.16 %
7.41 %
7.65 %
7.90 %
8.15 %
8.40 %
8.64 %
8.89 %
9.14 %
9.38 %
9.63 %
9.88 %
10.12 %
10.37 %
10.62 %
10.86 %
11.11 %
11.36 %
11.61 %
11.85 %
12.10 %
12.35 %
12.59 %
12.84 %
13.09 %
13.33 %
13.58 %
13.83 %
14.07 %
14.32 %
14.57 %
14.82 %
15.06 %
15.31 %
15.56 %
15.80 %
16.05 %
16.30 %
16.54 %
16.79 %
17.04 %
17.28 %
17.53 %
17.78 %
18.03 %
18.27 %
18.52 %
18.77 %
19.01 %
19.26 %
19.51 %
19.75 %
20.00 %
20.25 %
20.49 %
20.74 %
20.99 %
21.24 %
21.48 %
21.73 %
21.98 %
22.22 %
22.47 %
22.72 %
22.96 %
23.21 %
23.46 %
23.70 %
23.95 %
24.20 %
24.45 %
24.69 %
24.94 %
25.19 %
25.43 %
25.68 %
25.93 %
26.17 %
26.42 %
26.67 %
26.91 %
27.16 %
27.41 %
27.66 %
27.90 %
28.15 %
28.40 %
28.64 %
28.89 %
29.14 %
29.38 %
29.63 %
29.88 %
30.12 %
30.37 %
30.62 %
30.86 %
31.11 %
31.36 %
31.61 %
31.85 %
3

# Handling errors movies

I just restart the script the the error ones

In [36]:
logger.error('--New--')

In [37]:
# restart timeout
datas = []
with open("extration.log", "r") as f:
    txt = f.read().split('--New--')[-2]
    
missing = re.findall("(tt[0-9]{7})", txt)

for tconst in tqdm.tqdm(missing):
    content = extract_content(tconst, key)
    if content != -1 and content["Response"] == "True":
        datas.append(content)
df2 = pd.DataFrame.from_dict(datas)
df2.to_csv("../datas/raw/omdb_error2.csv", index=False)      

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:04<00:00,  3.03it/s]


# Merge dataset

To finish, let's merge all those dataset

In [62]:
list_df = []
for i, file in enumerate(glob.glob("../datas/raw/omdb_*.csv")):
    df = pd.read_csv(file, encoding="ISO-8859-1")
    if 'Episode' in df.columns:
        df.drop('Episode', axis=1, inplace= True)
    if 'seriesID' in df.columns:
        df.drop('seriesID', axis=1, inplace= True)
    if 'Season' in df.columns:
        df.drop('Season', axis=1, inplace= True)    
    list_df.append(df)
    
result = pd.concat(list_df)
result.to_csv("../datas/raw/full_omdb.csv", index=False) 